In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.impute import KNNImputer

import warnings
warnings.filterwarnings(action='ignore')

In [43]:
weather_2020 = pd.read_csv('2020_weather.csv', sep=',', encoding='cp949')
weather_2021 = pd.read_csv('2021_weather.csv', sep=',', encoding='cp949')
weather_2022 = pd.read_csv('2022_weather.csv', sep=',', encoding='cp949')

In [2]:
weather_2020

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),일조(hr),일사(MJ/m2),적설(cm),시정(10m),지면온도(°C)
0,108,서울,2020-01-01 00:00,-6.5,0.0,0.0,0.0,38,1.4,-18.5,1022.6,1033.8,NaN,NaN,NaN,2000.0,-2.8
1,108,서울,2020-01-01 01:00,-5.9,NaN,1.7,50.0,40,1.6,-17.3,1021.8,1033.0,NaN,NaN,NaN,2000.0,-2.4
2,108,서울,2020-01-01 02:00,-5.7,NaN,0.1,0.0,42,1.7,-16.5,1021.6,1032.8,NaN,NaN,NaN,1988.0,-2.4
3,108,서울,2020-01-01 03:00,-5.6,0.0,0.0,0.0,46,1.8,-15.4,1021.5,1032.7,NaN,NaN,NaN,2000.0,-2.7
4,108,서울,2020-01-01 04:00,-5.4,NaN,0.0,0.0,50,2.0,-14.2,1021.6,1032.8,NaN,NaN,NaN,1908.0,-2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17563,159,부산,2020-12-31 19:00,0.6,NaN,6.2,270.0,41,2.6,-11.1,1014.0,1022.8,NaN,NaN,NaN,5000.0,-0.5
17564,159,부산,2020-12-31 20:00,0.1,NaN,6.0,270.0,44,2.7,-10.7,1014.8,1023.6,NaN,NaN,NaN,5000.0,-0.8
17565,159,부산,2020-12-31 21:00,-0.2,NaN,5.0,290.0,48,2.9,-9.9,1014.6,1023.4,NaN,NaN,NaN,5000.0,-1.0
17566,159,부산,2020-12-31 22:00,-0.7,NaN,2.7,270.0,51,3.0,-9.6,1014.8,1023.6,NaN,NaN,NaN,5000.0,-1.3


In [3]:
weather_2020.columns

Index(['지점', '지점명', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)',
       '일사(MJ/m2)', '적설(cm)', '시정(10m)', '지면온도(°C)'],
      dtype='object')

In [44]:
busan_cols = []
seoul_cols = []

for i in weather_2020.columns:
    busan_cols.append('부산 '+i)
    seoul_cols.append('서울 '+i)
busan_cols, seoul_cols

(['부산 지점',
  '부산 지점명',
  '부산 일시',
  '부산 기온(°C)',
  '부산 강수량(mm)',
  '부산 풍속(m/s)',
  '부산 풍향(16방위)',
  '부산 습도(%)',
  '부산 증기압(hPa)',
  '부산 이슬점온도(°C)',
  '부산 현지기압(hPa)',
  '부산 해면기압(hPa)',
  '부산 일조(hr)',
  '부산 일사(MJ/m2)',
  '부산 적설(cm)',
  '부산 시정(10m)',
  '부산 지면온도(°C)'],
 ['서울 지점',
  '서울 지점명',
  '서울 일시',
  '서울 기온(°C)',
  '서울 강수량(mm)',
  '서울 풍속(m/s)',
  '서울 풍향(16방위)',
  '서울 습도(%)',
  '서울 증기압(hPa)',
  '서울 이슬점온도(°C)',
  '서울 현지기압(hPa)',
  '서울 해면기압(hPa)',
  '서울 일조(hr)',
  '서울 일사(MJ/m2)',
  '서울 적설(cm)',
  '서울 시정(10m)',
  '서울 지면온도(°C)'])

1. Weather train data

In [45]:
weather_data = pd.DataFrame()

for data in [weather_2020, weather_2021]:
    busan_weather = data.loc[data['지점']==159]
    busan_weather.reset_index(inplace=True, drop=True)
    busan_weather.columns = busan_cols
    
    seoul_weather = data.loc[data['지점']==108]
    seoul_weather.columns = seoul_cols
    
    new_data = pd.DataFrame(seoul_weather.loc[:, '서울 일시'])
    new_data.columns = ['일시']

    weather = pd.concat([seoul_weather.iloc[:, 3:], busan_weather.iloc[:, 3:]], axis=1)
    new_data = pd.concat([new_data, weather], axis=1)
    weather_data = pd.concat([weather_data, new_data], axis=0)

In [46]:
weather_data.to_csv('weather_data.csv', index=False)

In [29]:
weather_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),...,부산 습도(%),부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C)
0,2020-01-01 00:00,-6.5,0.0,0.0,0.0,38,1.4,-18.5,1022.6,1033.8,...,23,1.2,-20.8,1021.7,1030.7,NaN,NaN,NaN,2000.0,-1.4
1,2020-01-01 01:00,-5.9,NaN,1.7,50.0,40,1.6,-17.3,1021.8,1033.0,...,22,1.1,-21.7,1021.6,1030.6,NaN,NaN,NaN,2000.0,-1.6
2,2020-01-01 02:00,-5.7,NaN,0.1,0.0,42,1.7,-16.5,1021.6,1032.8,...,23,1.1,-21.3,1021.6,1030.6,NaN,NaN,NaN,2000.0,-1.9
3,2020-01-01 03:00,-5.6,0.0,0.0,0.0,46,1.8,-15.4,1021.5,1032.7,...,24,1.2,-20.6,1021.1,1030.1,NaN,NaN,NaN,2000.0,-2.0
4,2020-01-01 04:00,-5.4,NaN,0.0,0.0,50,2.0,-14.2,1021.6,1032.8,...,35,1.7,-16.4,1021.0,1030.0,NaN,NaN,NaN,2000.0,-2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2021-12-31 19:00,-6.4,NaN,2.2,250.0,34,1.3,-19.7,1021.1,1032.3,...,22,1.4,-18.8,1019.1,1028.0,NaN,NaN,NaN,5000.0,-1.6
8756,2021-12-31 20:00,-6.3,NaN,4.1,320.0,35,1.3,-19.2,1021.2,1032.4,...,22,1.3,-19.2,1019.1,1028.0,NaN,NaN,NaN,5000.0,-2.1
8757,2021-12-31 21:00,-6.7,NaN,4.8,320.0,36,1.3,-19.3,1021.2,1032.4,...,23,1.4,-19.0,1018.7,1027.6,NaN,NaN,NaN,5000.0,-2.5
8758,2021-12-31 22:00,-7.5,NaN,3.0,320.0,37,1.3,-19.7,1021.8,1033.1,...,23,1.3,-19.4,1019.5,1028.4,NaN,NaN,NaN,5000.0,-2.2


---
2. Weather test data

In [47]:
weather_test_data = pd.DataFrame()

busan_weather = weather_2022.loc[weather_2022['지점']==159]
busan_weather.reset_index(inplace=True, drop=True)
busan_weather.columns = busan_cols

seoul_weather = weather_2022.loc[weather_2022['지점']==108]
seoul_weather.columns = seoul_cols

new_data = pd.DataFrame(seoul_weather.loc[:, '서울 일시'])
new_data.columns = ['일시']

weather = pd.concat([seoul_weather.iloc[:, 3:], busan_weather.iloc[:, 3:]], axis=1)
new_data = pd.concat([new_data, weather], axis=1)
weather_test_data = pd.concat([weather_test_data, new_data], axis=0)

In [48]:
weather_test_data.to_csv('weather_test_data.csv', index=False)

---

3. time train data

In [49]:
time_data = pd.DataFrame()

for year in ['2020', '2021']:
    for month in range(1, 13):
        data = pd.read_csv(year+'_'+str(month)+'월.csv', sep=',', encoding='cp949')
        for i in range(len(data.columns)-1, 3, -1):
            new_data = data.iloc[:, i]
            time_data = pd.concat([time_data, new_data], axis=0)
time_data.columns = ['소요시간']

In [50]:
time_data

,소요시간
0,0
1,0
2,04:51
3,04:44
4,06:03
...,...
19,05:09
20,05:20
21,05:23
22,05:55


4. time test data

In [51]:
time_test_data = pd.DataFrame()

for month in range(1, 4):
    data = pd.read_csv('2022_'+str(month)+'월.csv', sep=',', encoding='cp949')
    for i in range(len(data.columns)-1, 3, -1):
        new_data = data.iloc[:, i]
        time_test_data = pd.concat([time_test_data, new_data], axis=0)
time_test_data.columns = ['소요시간']

In [52]:
time_test_data

,소요시간
0,03:47
1,03:56
2,0
3,03:53
4,03:49
...,...
19,04:23
20,04:29
21,04:25
22,05:31


In [53]:
weather_data.reset_index(inplace=True, drop=True)
weather_test_data.reset_index(inplace=True, drop=True)
time_data.reset_index(inplace=True, drop=True)
time_test_data.reset_index(inplace=True, drop=True)

In [54]:
train_data = weather_data.copy()
train_data = pd.concat([train_data, time_data], axis=1)
pd.set_option('display.max_columns', train_data.shape[0])
train_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),부산 강수량(mm),부산 풍속(m/s),부산 풍향(16방위),부산 습도(%),부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C),소요시간
0,2020-01-01 00:00,-6.5,0.0,0.0,0.0,38,1.4,-18.5,1022.6,1033.8,NaN,NaN,NaN,2000.0,-2.8,-2.5,NaN,1.7,360.0,23,1.2,-20.8,1021.7,1030.7,NaN,NaN,NaN,2000.0,-1.4,0
1,2020-01-01 01:00,-5.9,NaN,1.7,50.0,40,1.6,-17.3,1021.8,1033.0,NaN,NaN,NaN,2000.0,-2.4,-2.9,NaN,2.0,340.0,22,1.1,-21.7,1021.6,1030.6,NaN,NaN,NaN,2000.0,-1.6,0
2,2020-01-01 02:00,-5.7,NaN,0.1,0.0,42,1.7,-16.5,1021.6,1032.8,NaN,NaN,NaN,1988.0,-2.4,-3.1,NaN,1.9,140.0,23,1.1,-21.3,1021.6,1030.6,NaN,NaN,NaN,2000.0,-1.9,04:51
3,2020-01-01 03:00,-5.6,0.0,0.0,0.0,46,1.8,-15.4,1021.5,1032.7,NaN,NaN,NaN,2000.0,-2.7,-2.8,NaN,4.5,20.0,24,1.2,-20.6,1021.1,1030.1,NaN,NaN,NaN,2000.0,-2.0,04:44
4,2020-01-01 04:00,-5.4,NaN,0.0,0.0,50,2.0,-14.2,1021.6,1032.8,NaN,NaN,NaN,1908.0,-2.5,-3.1,NaN,1.9,110.0,35,1.7,-16.4,1021.0,1030.0,NaN,NaN,NaN,2000.0,-2.1,06:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17539,2021-12-31 19:00,-6.4,NaN,2.2,250.0,34,1.3,-19.7,1021.1,1032.3,NaN,NaN,NaN,2000.0,-4.5,0.5,NaN,2.0,320.0,22,1.4,-18.8,1019.1,1028.0,NaN,NaN,NaN,5000.0,-1.6,05:09
17540,2021-12-31 20:00,-6.3,NaN,4.1,320.0,35,1.3,-19.2,1021.2,1032.4,NaN,NaN,NaN,2000.0,-5.3,0.0,NaN,2.2,320.0,22,1.3,-19.2,1019.1,1028.0,NaN,NaN,NaN,5000.0,-2.1,05:20
17541,2021-12-31 21:00,-6.7,NaN,4.8,320.0,36,1.3,-19.3,1021.2,1032.4,NaN,NaN,NaN,2000.0,-5.7,-0.4,NaN,4.1,290.0,23,1.4,-19.0,1018.7,1027.6,NaN,NaN,NaN,5000.0,-2.5,05:23
17542,2021-12-31 22:00,-7.5,NaN,3.0,320.0,37,1.3,-19.7,1021.8,1033.1,NaN,NaN,NaN,2000.0,-6.2,-0.8,NaN,3.2,250.0,23,1.3,-19.4,1019.5,1028.4,NaN,NaN,NaN,5000.0,-2.2,05:55


In [38]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17544 entries, 0 to 17543
Data columns (total 30 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   일시            17544 non-null  object 
 1   서울 기온(°C)     17543 non-null  float64
 2   서울 강수량(mm)    2010 non-null   float64
 3   서울 풍속(m/s)    17542 non-null  float64
 4   서울 풍향(16방위)   17542 non-null  float64
 5   서울 습도(%)      17544 non-null  int64  
 6   서울 증기압(hPa)   17542 non-null  float64
 7   서울 이슬점온도(°C)  17537 non-null  float64
 8   서울 현지기압(hPa)  17531 non-null  float64
 9   서울 해면기압(hPa)  17531 non-null  float64
 10  서울 일조(hr)     9593 non-null   float64
 11  서울 일사(MJ/m2)  9593 non-null   float64
 12  서울 적설(cm)     587 non-null    float64
 13  서울 시정(10m)    17544 non-null  float64
 14  서울 지면온도(°C)   17531 non-null  float64
 15  부산 기온(°C)     17539 non-null  float64
 16  부산 강수량(mm)    1820 non-null   float64
 17  부산 풍속(m/s)    17517 non-null  float64
 18  부산 풍향(16방위)   17517 non-nu

In [55]:
test_data = weather_test_data.copy()
test_data = pd.concat([test_data, time_test_data], axis=1)
pd.set_option('display.max_columns', train_data.shape[0])
test_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),부산 강수량(mm),부산 풍속(m/s),부산 풍향(16방위),부산 습도(%),부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C),소요시간
0,2022-01-01 00:00,-8.5,NaN,1.9,320,41,1.3,-19.4,1021.9,1033.2,NaN,NaN,NaN,2000,-7.0,-1.6,NaN,7.2,290,25,1.4,-19.1,1019.7,1028.6,NaN,NaN,NaN,5000,-4.4,03:47
1,2022-01-01 01:00,-9.2,NaN,1.8,270,42,1.3,-19.7,1022.2,1033.6,NaN,NaN,NaN,2000,-7.2,-2.0,NaN,3.6,200,24,1.3,-19.9,1019.9,1028.8,NaN,NaN,NaN,5000,-4.7,03:56
2,2022-01-01 02:00,-9.5,NaN,1.2,270,43,1.3,-19.7,1022.3,1033.7,NaN,NaN,NaN,2000,-7.5,-2.3,NaN,2.4,230,24,1.2,-20.1,1020.3,1029.3,NaN,NaN,NaN,5000,-4.7,0
3,2022-01-01 03:00,-9.3,NaN,1.4,290,46,1.4,-18.8,1022.6,1034.0,NaN,NaN,NaN,2000,-7.6,-2.8,NaN,3.5,270,27,1.3,-19.2,1020.6,1029.6,NaN,NaN,NaN,5000,-5.3,03:53
4,2022-01-01 04:00,-9.6,NaN,1.7,340,48,1.4,-18.5,1021.9,1033.3,NaN,NaN,NaN,2000,-7.6,-2.3,NaN,2.4,230,20,1.0,-22.2,1020.0,1029.0,NaN,NaN,NaN,5000,-5.5,03:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2022-03-31 19:00,12.3,NaN,2.9,340,47,6.7,1.2,1013.4,1023.8,0.4,0.2,NaN,2000,10.0,10.0,NaN,4.3,50,77,9.4,6.1,1014.8,1023.3,0.0,0.02,NaN,3987,10.8,04:23
2156,2022-03-31 20:00,11.6,NaN,2.7,340,48,6.5,0.9,1014.6,1025.0,NaN,NaN,NaN,2000,8.9,9.3,NaN,4.0,50,87,10.2,7.2,1015.7,1024.2,NaN,NaN,NaN,1144,9.7,04:29
2157,2022-03-31 21:00,10.5,NaN,3.5,320,51,6.4,0.7,1015.4,1025.9,NaN,NaN,NaN,2000,7.8,9.1,0.0,2.8,50,78,9.0,5.4,1016.6,1025.1,NaN,NaN,NaN,4489,9.3,04:25
2158,2022-03-31 22:00,9.3,NaN,1.9,50,58,6.8,1.4,1016.4,1026.9,NaN,NaN,NaN,2000,7.3,9.0,NaN,3.8,50,73,8.4,4.4,1017.1,1025.6,NaN,NaN,NaN,5000,9.1,05:31


In [56]:
train_data['서울 풍향(16방위)'] = np.radians(train_data['서울 풍향(16방위)'])
train_data['부산 풍향(16방위)'] = np.radians(train_data['부산 풍향(16방위)'])
test_data['서울 풍향(16방위)'] = np.radians(test_data['서울 풍향(16방위)'])
test_data['부산 풍향(16방위)'] = np.radians(test_data['부산 풍향(16방위)'])

In [60]:
train_data.isna().sum()

일시                  0
서울 기온(°C)           1
서울 강수량(mm)      15534
서울 풍속(m/s)          2
서울 풍향(16방위)         2
서울 습도(%)            0
서울 증기압(hPa)         2
서울 이슬점온도(°C)        7
서울 현지기압(hPa)       13
서울 해면기압(hPa)       13
서울 일조(hr)        7951
서울 일사(MJ/m2)     7951
서울 적설(cm)       16957
서울 시정(10m)          0
서울 지면온도(°C)        13
부산 기온(°C)           5
부산 강수량(mm)      15724
부산 풍속(m/s)         27
부산 풍향(16방위)        27
부산 습도(%)            0
부산 증기압(hPa)         4
부산 이슬점온도(°C)        6
부산 현지기압(hPa)        4
부산 해면기압(hPa)        1
부산 일조(hr)        7955
부산 일사(MJ/m2)     7962
부산 적설(cm)       17544
부산 시정(10m)          8
부산 지면온도(°C)         0
소요시간                0
dtype: int64

In [59]:
test_data.columns

Index(['일시', '서울 기온(°C)', '서울 강수량(mm)', '서울 풍속(m/s)', '서울 풍향(16방위)',
       '서울 습도(%)', '서울 증기압(hPa)', '서울 이슬점온도(°C)', '서울 현지기압(hPa)',
       '서울 해면기압(hPa)', '서울 일조(hr)', '서울 일사(MJ/m2)', '서울 적설(cm)', '서울 시정(10m)',
       '서울 지면온도(°C)', '부산 기온(°C)', '부산 강수량(mm)', '부산 풍속(m/s)', '부산 풍향(16방위)',
       '부산 습도(%)', '부산 증기압(hPa)', '부산 이슬점온도(°C)', '부산 현지기압(hPa)',
       '부산 해면기압(hPa)', '부산 일조(hr)', '부산 일사(MJ/m2)', '부산 적설(cm)', '부산 시정(10m)',
       '부산 지면온도(°C)', '소요시간'],
      dtype='object')

In [61]:
cols = ['서울 강수량(mm)', '서울 일조(hr)', '서울 일사(MJ/m2)', '서울 적설(cm)', '부산 강수량(mm)', '부산 일조(hr)', '부산 일사(MJ/m2)', '부산 적설(cm)']
for col in cols:
    train_data[col].fillna(0, inplace=True)
    test_data[col].fillna(0, inplace=True)

In [62]:
train_data.isna().sum()

일시               0
서울 기온(°C)        1
서울 강수량(mm)       0
서울 풍속(m/s)       2
서울 풍향(16방위)      2
서울 습도(%)         0
서울 증기압(hPa)      2
서울 이슬점온도(°C)     7
서울 현지기압(hPa)    13
서울 해면기압(hPa)    13
서울 일조(hr)        0
서울 일사(MJ/m2)     0
서울 적설(cm)        0
서울 시정(10m)       0
서울 지면온도(°C)     13
부산 기온(°C)        5
부산 강수량(mm)       0
부산 풍속(m/s)      27
부산 풍향(16방위)     27
부산 습도(%)         0
부산 증기압(hPa)      4
부산 이슬점온도(°C)     6
부산 현지기압(hPa)     4
부산 해면기압(hPa)     1
부산 일조(hr)        0
부산 일사(MJ/m2)     0
부산 적설(cm)        0
부산 시정(10m)       8
부산 지면온도(°C)      0
소요시간             0
dtype: int64

In [63]:
from sklearn.preprocessing import LabelEncoder

cols = ['서울 풍향(16방위)','부산 풍향(16방위)']
for feature in cols:
    # LabelEncoder를 객체로 생성한 후 , fit( ) 과 transform( ) 으로 label 인코딩 수행. 
    encoder = LabelEncoder()
    encoder.fit(train_data[feature])
    train_data[feature] = encoder.transform(train_data[feature])
    
    encoder.fit(test_data[feature])
    test_data[feature] = encoder.transform(test_data[feature])

In [64]:
train_data['소요시간'] = train_data['소요시간'].replace('0', np.NaN)
test_data['소요시간'] = test_data['소요시간'].replace('0', np.NaN)

In [65]:
for index, row in train_data.iterrows():
    tmp = row['소요시간']
    if ':' not in str(tmp):
        continue
    h, m = str(tmp).split(':')
    row['소요시간'] = int(h)*60+int(m)
    train_data.iloc[index] = row

In [67]:
train_data['소요시간'] = train_data['소요시간'].astype(float)

In [66]:
for index, row in test_data.iterrows():
    tmp = row['소요시간']
    if ':' not in str(tmp):
        continue
    h, m = str(tmp).split(':')
    row['소요시간'] = int(h)*60+int(m)
    test_data.iloc[index] = row

In [68]:
test_data['소요시간'] = test_data['소요시간'].astype(float)

In [69]:
tmp_train = train_data.iloc[:, 1:]
tmp_test = test_data.iloc[:, 1:]

In [72]:
imputer = KNNImputer()
tmp_train = pd.DataFrame(imputer.fit_transform(tmp_train), columns=tmp_train.columns)
tmp_test = pd.DataFrame(imputer.fit_transform(tmp_test), columns=tmp_test.columns)

In [73]:
for feature in train_data.columns[1:]:
    train_data[feature] = tmp_train[feature]
    test_data[feature] = tmp_test[feature]

In [77]:
train_data['소요시간'] = train_data['소요시간'].astype(int)
test_data['소요시간'] = test_data['소요시간'].astype(int)

In [78]:
train_data.isna().sum()

일시              0
서울 기온(°C)       0
서울 강수량(mm)      0
서울 풍속(m/s)      0
서울 풍향(16방위)     0
서울 습도(%)        0
서울 증기압(hPa)     0
서울 이슬점온도(°C)    0
서울 현지기압(hPa)    0
서울 해면기압(hPa)    0
서울 일조(hr)       0
서울 일사(MJ/m2)    0
서울 적설(cm)       0
서울 시정(10m)      0
서울 지면온도(°C)     0
부산 기온(°C)       0
부산 강수량(mm)      0
부산 풍속(m/s)      0
부산 풍향(16방위)     0
부산 습도(%)        0
부산 증기압(hPa)     0
부산 이슬점온도(°C)    0
부산 현지기압(hPa)    0
부산 해면기압(hPa)    0
부산 일조(hr)       0
부산 일사(MJ/m2)    0
부산 적설(cm)       0
부산 시정(10m)      0
부산 지면온도(°C)     0
소요시간            0
dtype: int64

In [79]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

### 공휴일, 요일, 명절여부 추가하기

1. 일시 -> Datetime

In [81]:
train_data

,일시,서울 기온(°C),서울 강수량(mm),서울 풍속(m/s),서울 풍향(16방위),서울 습도(%),서울 증기압(hPa),서울 이슬점온도(°C),서울 현지기압(hPa),서울 해면기압(hPa),서울 일조(hr),서울 일사(MJ/m2),서울 적설(cm),서울 시정(10m),서울 지면온도(°C),부산 기온(°C),부산 강수량(mm),부산 풍속(m/s),부산 풍향(16방위),부산 습도(%),부산 증기압(hPa),부산 이슬점온도(°C),부산 현지기압(hPa),부산 해면기압(hPa),부산 일조(hr),부산 일사(MJ/m2),부산 적설(cm),부산 시정(10m),부산 지면온도(°C),소요시간
0,2020-01-01 00:00,-6.5,0.0,0.0,0.0,38.0,1.4,-18.5,1022.6,1033.8,0.0,0.0,0.0,2000.0,-2.8,-2.5,0.0,1.7,16.0,23.0,1.2,-20.8,1021.7,1030.7,0.0,0.0,0.0,2000.0,-1.4,265
1,2020-01-01 01:00,-5.9,0.0,1.7,2.0,40.0,1.6,-17.3,1021.8,1033.0,0.0,0.0,0.0,2000.0,-2.4,-2.9,0.0,2.0,15.0,22.0,1.1,-21.7,1021.6,1030.6,0.0,0.0,0.0,2000.0,-1.6,265
2,2020-01-01 02:00,-5.7,0.0,0.1,0.0,42.0,1.7,-16.5,1021.6,1032.8,0.0,0.0,0.0,1988.0,-2.4,-3.1,0.0,1.9,6.0,23.0,1.1,-21.3,1021.6,1030.6,0.0,0.0,0.0,2000.0,-1.9,291
3,2020-01-01 03:00,-5.6,0.0,0.0,0.0,46.0,1.8,-15.4,1021.5,1032.7,0.0,0.0,0.0,2000.0,-2.7,-2.8,0.0,4.5,1.0,24.0,1.2,-20.6,1021.1,1030.1,0.0,0.0,0.0,2000.0,-2.0,284
4,2020-01-01 04:00,-5.4,0.0,0.0,0.0,50.0,2.0,-14.2,1021.6,1032.8,0.0,0.0,0.0,1908.0,-2.5,-3.1,0.0,1.9,5.0,35.0,1.7,-16.4,1021.0,1030.0,0.0,0.0,0.0,2000.0,-2.1,363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17539,2021-12-31 19:00,-6.4,0.0,2.2,11.0,34.0,1.3,-19.7,1021.1,1032.3,0.0,0.0,0.0,2000.0,-4.5,0.5,0.0,2.0,14.0,22.0,1.4,-18.8,1019.1,1028.0,0.0,0.0,0.0,5000.0,-1.6,309
17540,2021-12-31 20:00,-6.3,0.0,4.1,14.0,35.0,1.3,-19.2,1021.2,1032.4,0.0,0.0,0.0,2000.0,-5.3,0.0,0.0,2.2,14.0,22.0,1.3,-19.2,1019.1,1028.0,0.0,0.0,0.0,5000.0,-2.1,320
17541,2021-12-31 21:00,-6.7,0.0,4.8,14.0,36.0,1.3,-19.3,1021.2,1032.4,0.0,0.0,0.0,2000.0,-5.7,-0.4,0.0,4.1,13.0,23.0,1.4,-19.0,1018.7,1027.6,0.0,0.0,0.0,5000.0,-2.5,323
17542,2021-12-31 22:00,-7.5,0.0,3.0,14.0,37.0,1.3,-19.7,1021.8,1033.1,0.0,0.0,0.0,2000.0,-6.2,-0.8,0.0,3.2,11.0,23.0,1.3,-19.4,1019.5,1028.4,0.0,0.0,0.0,5000.0,-2.2,355


In [83]:
train_data['일시'] = pd.to_datetime(train_data['일시'], format='%Y-%m-%d %H:%M', errors='raise')
test_data['일시'] = pd.to_datetime(test_data['일시'], format='%Y-%m-%d %H:%M', errors='raise')

2. 요일 추가

In [98]:
train_data['weekday'] = train_data['일시'].dt.day_name()
train_data['weekday'] = train_data['weekday'].str.slice(start=0, stop=3)
test_data['weekday'] = test_data['일시'].dt.day_name()
test_data['weekday'] = test_data['weekday'].str.slice(start=0, stop=3)

2. 공휴일 추가

In [103]:
train_data['holiday'] = 0
test_data['holiday'] = 0

In [105]:
month_2020 = [1, 1, 1, 1, 3, 4, 5, 6, 8, 8, 9, 10, 10, 10, 10, 12]
day_2020 = [1, 24, 25, 26, 1, 30, 5, 6, 15, 17, 30, 1, 2, 3, 9, 25]

for i in range(len(month_2020)):
    m = month_2020[i]
    d = day_2020[i]
    
    train_data.loc[(train_data['일시'].dt.year==2020) & (train_data['일시'].dt.month==m) & (train_data['일시'].dt.day==d), 'holiday'] = 1

In [106]:
month_2021 = [1, 2, 2, 2, 3, 5, 5, 6, 8, 9, 9, 9, 10, 10, 12]
day_2021 = [1, 11, 12, 13, 1, 5, 19, 6, 15, 20, 21, 22, 3, 9, 25]

for i in range(len(month_2021)):
    m = month_2021[i]
    d = day_2021[i]
    
    train_data.loc[(train_data['일시'].dt.year==2021) & (train_data['일시'].dt.month==m) & (train_data['일시'].dt.day==d), 'holiday'] = 1

In [107]:
month_2022 = [1, 1, 2, 2, 3]
day_2022 = [1, 31, 1, 2, 1]

for i in range(len(month_2022)):
    m = month_2022[i]
    d = day_2022[i]
    
    test_data.loc[(test_data['일시'].dt.year==2022) & (test_data['일시'].dt.month==m) & (test_data['일시'].dt.day==d), 'holiday'] = 1

3. 명절 추가

In [109]:
train_data['Korean_holiday'] = 0
test_data['Korean_holiday'] = 0

In [110]:
year = [2020, 2020, 2020, 2020, 2020, 2020,
        2021, 2021, 2021, 2021, 2021, 2021]

month = [1, 1, 1, 9, 10, 10,
         2, 2, 2, 9, 9, 9]
month2 = [1, 2, 2]

day = [24, 25, 26, 30, 1, 2,
       11, 12, 13, 20, 21, 22]
day2 = [31, 1, 2]

for i in range(len(year)):
    y = year[i]
    m = month[i]
    d = day[i]
    
    train_data.loc[(train_data['일시'].dt.year==y) & (train_data['일시'].dt.month==m) & (train_data['일시'].dt.day==d), 'Korean_holiday'] = 1
    
for i in range(3):
    m = month2[i]
    d = day2[i]
    
    test_data.loc[(test_data['일시'].dt.year==2022) & (test_data['일시'].dt.month==m) & (test_data['일시'].dt.day==d), 'Korean_holiday'] = 1

In [113]:
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

4. year, month 추가

In [112]:
train_data['year'] = train_data['일시'].dt.year
train_data['month'] = train_data['일시'].dt.month
test_data['year'] = test_data['일시'].dt.year
test_data['month'] = test_data['일시'].dt.month

In [114]:
test_data.shape

(2160, 35)